# İş’i Anlamak (Business Understanding)

**Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama**


Bir oyun şirketi müşterilerinin bazı özelliklerini kullanarak
seviye tabanlı (level based) yeni müşteri tanımları (persona)
oluşturmak ve bu yeni müşteri tanımlarına göre segmentler
oluşturup bu segmentlere göre yeni gelebilecek müşterilerin
şirkete ortalama ne kadar kazandırabileceğini tahmin etmek
istemektedir.

Örneğin:
Türkiye’den IOS kullanıcısı olan 25 yaşındaki bir erkek
kullanıcının ortalama ne kadar kazandırabileceği belirlenmek
isteniyor.




# Veriyi Anlamak (Data Understanding)

Persona.csv veri seti uluslararası bir oyun şirketinin sattığı ürünlerin fiyatlarını ve bu
ürünleri satın alan kullanıcıların bazı demografik bilgilerini barındırmaktadır. Veri
seti her satış işleminde oluşan kayıtlardan meydana gelmektedir. Bunun anlamı
tablo tekilleştirilmemiştir. Diğer bir ifade ile belirli demografik özelliklere sahip bir
kullanıcı birden fazla alışveriş yapmış olabilir

# Veriyi Hazırlamak (Data Preparation)

**Veri Setimiz hazır geldiği için bir değişiklik şu an söz konusu değildir.**

* PRICE : Müşterinin harcama tutarı
* SOURCE : Müşterinin bağlandığı cihaz türü
* SEX : Müşterinin cinsiyeti
* COUNTRY : Müşterinin ülkesi
* AGE : Müşterinin yaşı 

In [204]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.set_option("display.max_rows",None)  #veri setinin okunaklı olması için

In [205]:
#persona.csv dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.

data = pd.read_csv("persona.csv")
df = data.copy()

In [206]:
#veri setinin ilk ve son 10 değerini inceliyoruz
df.head(10)
df.tail(10)

,PRICE,SOURCE,SEX,COUNTRY,AGE
4990,39,ios,male,usa,27
4991,29,ios,male,usa,27
4992,19,ios,male,usa,27
4993,39,ios,male,usa,27
4994,39,android,female,bra,31
4995,29,android,female,bra,31
4996,29,android,female,bra,31
4997,29,android,female,bra,31
4998,39,android,female,bra,31
4999,29,android,female,bra,31


In [207]:
df.shape

#veri setimiz 5000 satırdan ve 5 sütundan oluşmaktadır

(5000, 5)

In [208]:
df.info()

#veri setinde null değerimiz yok
## object olarak tutulan değerlere dikkat ediyoruz!

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [209]:
#Kaç unique SOURCE vardır? Frekansları nedir?

df["SOURCE"].nunique()  ## 2 adet var => android ve ios olmak üzere
df["SOURCE"].value_counts() ## android ve ios cihazlarn. sayısı

android    2974
ios        2026
Name: SOURCE, dtype: int64

In [210]:
# Price değşk. segmentler görünüyor, bunları incelemekte fayda var

df["PRICE"].nunique()  # 6 adet fiyat segmenti görünüyor
df["PRICE"].value_counts() # hangi segmentten kaç adet olduğuna bakalım

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

In [211]:
#Hangi ülkeden kaç adet satış gerçekleştirilmiştir

df["COUNTRY"].value_counts()

#en çok satış USA (2065) ve BRA'da (1496) gerçekleşmiş
#bu ülkeleri en yakın takip eden 3. ülke ise 455 satış ile DEU

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

In [212]:
#Peki ülkere göre satışlardan toplam kazanç nedir?
#Persona oluştururken bunlara dikkat edeceğiz  (potansiyel hedef için)

df.groupby("COUNTRY").agg("PRICE").sum()

#anomali yok 

COUNTRY
bra    51354
can     7730
deu    15485
fra    10177
tur    15689
usa    70225
Name: PRICE, dtype: int64

In [213]:
#Ülkelere göre PRICE ortalamaları nedir?
df.groupby("COUNTRY").agg("PRICE").mean()


# toplam kazanç ve toplam satış adetleri incelendiğinde orantısı doğru 

COUNTRY
bra    34.327540
can    33.608696
deu    34.032967
fra    33.587459
tur    34.787140
usa    34.007264
Name: PRICE, dtype: float64

In [214]:
#SOURCE'lara göre PRICE ortalamaları nedir?
df.groupby(by=["SOURCE"]).agg({"PRICE" : "mean"})

# fiyat ortalaması birbirine yakın görünüyor fakat 
# yüzdelik dilimde android kullanıcısı daha çoktur
# a.ortalama ve medyan arasındaki fark göz önünde bulundurulmalıdır. 

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [215]:
# Soru 10: COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?

df.groupby(["COUNTRY", "SOURCE"]).agg("PRICE").mean()

COUNTRY  SOURCE 
bra      android    34.387029
         ios        34.222222
can      android    33.330709
         ios        33.951456
deu      android    33.869888
         ios        34.268817
fra      android    34.312500
         ios        32.776224
tur      android    36.229437
         ios        33.272727
usa      android    33.760357
         ios        34.371703
Name: PRICE, dtype: float64

In [216]:
#COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir


df.groupby(["COUNTRY","SOURCE", "SEX", "AGE"]).agg({"PRICE": "mean"}).head()


PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897

In [217]:
#Çıktının düzgün okunması için sort_values metodunu azalan olacak şekilde PRICE'a uyguluyoruz.
#Çıktıyı agg_df olarak kaydediyoruz.

agg_df = df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"]).agg({"PRICE" : "mean"}).sort_values("PRICE", ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [218]:
#Indexleri düzenleyeceğiz (diger cıktıda index yok)

agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


In [219]:
#AGE değişkenini kategorik değişkene çevireceğiz ve agg_df'e ekleyerek bütün oluşturacağız.
#bu işlemleri gerçekleştirirken personalar için aralıklar ve etiketler belirleyeceğiz.

agg_df["AGE"].describe()

myBins = [0, 18, 23, 30, 40, agg_df["AGE"].max()]  #bins = aralık

myLabels = ["0_18", "19_23", "24_30", "31_40", "41 -" + str(agg_df["AGE"].max())] #labels = etiket, isimlendirme

agg_df["age_cat"] = pd.cut(agg_df["AGE"], bins = myBins, labels = myLabels)





agg_df.head()

# verileri gruplandırmak oldukça önemlidir, 
## bu yüzden yaş aralıklarına göre bir gruplandırma gerçekleştirdik
### bu sayede daha iyi bir gözlem gerçekleşir

,COUNTRY,SOURCE,SEX,AGE,PRICE,age_cat
0,bra,android,male,46,59.0,41 -66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


In [220]:
#customer_level_based ile değerleri oluşturacağız ve bu değişkene atayacağız

"""   

Dikkat! list comp customer_level_based değerleri oluşturulduktan sonra bu değerlerin tekilleştirilmesi gerekmektedir
Aksi durumda "ÇOKLAMA" gerçekleşir ve bu durum veriyi gözlemlemeyi zorlaştırır, veriyi manipüle eder
örn. birden fazla USA_ANDROID_MALE_0_18 değeri olabilir
Bu durumu halletmek için groupby kullanacağız    


"""
 
agg_df["customers_level_based"] = agg_df[["COUNTRY", "SOURCE", "SEX", "age_cat"]].agg(lambda x: "_".join(x).upper(), axis=1)
agg_df = agg_df[["customers_level_based", "PRICE"]]
agg_df = agg_df.groupby("customers_level_based").agg({"PRICE": "mean"}).reset_index()
agg_df.head()


,customers_level_based,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41 -66,36.737179


In [226]:
# Yeni müşterileri (USA_ANDROID_MALE_0_18) segmentlere ayıralım

# PRICE'a göre segmentlere ayır
# segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekle
# segmentleri betimle


agg_df["SEGMENT"] = pd.qcut (agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head (30)
agg_df.groupby("SEGMENT").agg({"PRICE": "mean"})

,PRICE
SEGMENT,
D,29.206780
C,33.509674
B,34.999645
A,38.691234


In [233]:
# Yeni gelen müşterileri sınıflandır ve  ne kadar gelir getirebileceğini tahminle
## 33 yaşında ANDROID kullanan bir Türk kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir? 

new_user = "TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]


,index,customers_level_based,PRICE,SEGMENT
72,72,TUR_ANDROID_FEMALE_31_40,41.833333,A


In [234]:
# 35 yaşında IOS kullanan bir Fransız kadını hangi segmente ve ortalama ne kadar gelir kazandırması beklenir? 

new_user = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

,index,customers_level_based,PRICE,SEGMENT
63,63,FRA_IOS_FEMALE_31_40,32.818182,C
